In [ ]:
#| default_exp routes.dataflow_routes

In [ ]:
#| exporti
import httpx
import pandas as pd
import io
from pprint import pprint

import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
#| export
async def get_dataflow_by_id(dataflow_id: int,
                            auth: dmda.DomoAuth,
                            debug_api: bool = False) -> rgd.ResponseGetData:
    domo_instance = auth.domo_instance

    url = f'https://{domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}'

    return await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
    )




#### Sample implementation of get_dataset by ID

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_dataflow_by_id(dataflow_id = 102, auth=token_auth)
res.response.keys()

dict_keys(['id', 'name', 'dapDataFlowId', 'responsibleUserId', 'runState', 'onboardFlowVersion', 'lastExecution', 'created', 'modified', 'actions', 'engineProperties', 'inputs', 'outputs', 'executionCount', 'executionSuccessCount', 'hydrationState', 'useLegacyTriggerBehavior', 'passwordProtected', 'deleted', 'abandoned', 'neverAbandon', 'settings', 'paused', 'enabled', 'editable', 'restricted', 'container', 'databaseType', 'triggeredByInput', 'draft', 'numInputs', 'numOutputs', 'magic'])